<a href="https://colab.research.google.com/github/kimx1201/movie_for_mood/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%B8%94%EB%A1%9C%EA%B7%B8_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import numpy as np
import html5lib
from selenium.webdriver.common.keys import Keys
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/project6/KOBIS_개봉일람.xlsx')

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
df = df[df['전국관객수'] > 1000000]

In [ ]:
df

In [ ]:
list(df['영화명'])[0]

In [ ]:
query_txt = list(df['영화명'])
start_date="20000101"
end_date="2021610"
q = '"영화"'

In [ ]:
path = 'chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')    # 언어 설정, options=options
options.add_argument('no-sandbox')
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
driver.get('http://www.naver.com')
time.sleep(2)
# 네이버 검색창에 검색어 검색
element = driver.find_element_by_id('query')
element.send_keys("영화", list(df['영화명'])[0])  # query_txt는 위에서 입력한 '영화 리뷰'
element.submit()

# 블로그 카테고리 선택
driver.find_element_by_link_text('블로그').click()

#검색 

driver.find_element_by_xpath('//*[@id="_search_option_btn"]').click()
time.sleep(1)

#날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

#날짜 입력
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear()  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
    
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)

# 종료 날짜 입력
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)

driver.find_element_by_class_name('tx').click()
time.sleep(3)
    
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()
time.sleep(1)
    
    
url_list = []
title_list = []
title_list2 = []
# 몇개의 페이지를 크롤링 할지 선택

total_page = 1
for j in tqdm(list(df['영화명'])[0:]) : 
    for i in range(total_page) : # 페이지 번호
        i = i * 1 
        url = "https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{2}to{3}&post_blogurl=&post_blogurl_without=&query=영화%20{4}%20줄거리&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date, j, i)
        driver.get(url)
        time.sleep(0.5)
        
# URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    moive_raw = driver.find_elements_by_css_selector(titles)
    count = 0
    # url 크롤링 시작    
    for movie in moive_raw[:4]: # url 4개만 긁음
        url = movie.get_attribute('href')
        url_list.append(url)
        count += 1
        if url == '' :
          url_list.append('0')
          print('0개')
        else :
            url_list.append(url)
            print(count, url)
     # 제목   
    for movie in moive_raw[:4]: # 타이틀도 4개만 긁어옴
        title = movie.get_attribute('title')   
        title_list2.append(title)    

        
for i in list(df['영화명'])[0:] :
    for j in range(1, 5) :
        title_list.append(i)

    
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))



df2 = pd.DataFrame({'url':url_list, 'title':title_list2})

# # 저장하기
df2.to_csv("blog_url2.csv", encoding='utf-8-sig')